In [11]:
import pystac
from pystac_client import Client
from shapely.geometry import Point
import rioxarray
from pyproj import CRS
import geopandas as gpd
from folium import Map, TileLayer
import numpy as np
import xarray
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import dask
import rasterio
from rasterio.plot import show
import json
from datetime import datetime
import pandas as pd
import httpx
from shapely.geometry import Polygon, mapping
import pycountry
import folium
from pygeotile.point import Point
import numpy as np
import math
import re

In [12]:
#titiler_endpoint = "https://titiler.xyz"
titiler_endpoint = "http://127.0.0.1:8000"

#url = "file:///Users/sashkawarner/repos/flood-data-api/app/data/B03.tiff"
url = "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/2/L/MK/2023/2/S2B_2LMK_20230225_0_L2A/B03.tif"
r = httpx.get(
    f"{titiler_endpoint}/tilejson.json",
    params = {
        #"country": "afghanistan",
        "url": url,
        #"assets": ["green", "swir16"],
        #"assets": ["green", "blue"],
        'minzoom': 8,
        'maxzoom': 16,
        "bidx": 1,
        "rescale": "0, 25",
        "colormap_name": "viridis",
        #"algorithm": "MNDWI",
        #"buffer": 3,
    }
).json()
r
#test_url = r['tiles'][0]
#deg_y, deg_x, zoom = r['center']
#print(test_url)
#print(deg_x, deg_y, zoom)

{'detail': 'assets must be defined either via expression or assets options.'}

In [ ]:
titiler_endpoint = "http://127.0.0.1:8000"
#url = "file:///Users/sashkawarner/repos/flood-data-api/app/data/test_mosaic.json"
#url = "/Users/sashkawarner/repos/flood-data-api/app/data/test_mosaic.json"
url = "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/2/L/MK/2023/2/S2B_2LMK_20230225_0_L2A"
#mosaic_test = json.load(open("../app/data/test_mosaic.json"))
r = httpx.get(
    url=f"{titiler_endpoint}/tilejson.json",
    params = {
        "url": url,
        'minzoom': 8,
        'maxzoom': 16,
        #"rescale": "0, 25",
        #"colormap_name": "viridis",
        "assets": ["green", "swir16"],
        "algorithm": "detectFlood",
        #"buffer": 0.5,
   }
).json()

r

In [25]:
titiler_endpoint = "http://127.0.0.1:8000"
#url = "file:///Users/sashkawarner/repos/flood-data-api/app/data/test_mosaic.json"
#url = "/Users/sashkawarner/repos/flood-data-api/app/data/test_mosaic.json"
#url = "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/2/L/MK/2023/2/S2B_2LMK_20230225_0_L2A"
#mosaic_test = json.load(open("../app/data/test_mosaic.json"))
r = httpx.get(
    url=f"{titiler_endpoint}/search",
    follow_redirects=True,
    params = {
        "country": "american SamOA",
   }
).json()

r

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['http://127.0.0.1:8000/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?url=https%3A%2F%2Fearth-search.aws.element84.com%2Fv0%2Fcollections%2Fsentinel-s2-l2a-cogs%2Fitems%2FS2A_2LQJ_20230227_0_L2A&assets=B03&assets=B11&algorithm=detectFlood&rescale=-1%2C1&colormap_name=viridis'],
 'minzoom': 8,
 'maxzoom': 14,
 'bounds': [-168.65178856189547,
  -15.1700621933571,
  -168.117486492895,
  -14.454757819240648],
 'center': [-168.38463752739523, -14.812410006298874, 8]}

In [26]:
test_url = r['tiles'][0]
deg_y, deg_x, zoom = r['center']
#print(test_url)
print(deg_x, deg_y, zoom)

def deg2num(lat_deg, lon_deg, zoom):
  lat_rad = math.radians(lat_deg)
  n = 2.0 ** zoom
  xtile = int((lon_deg + 180.0) / 360.0 * n)
  ytile = int((1.0 - math.asinh(math.tan(lat_rad)) / math.pi) / 2.0 * n)
  return (xtile, ytile)

tile_x, tile_y = deg2num(lat_deg=deg_x, lon_deg=deg_y, zoom=zoom)
print(tile_x, tile_y)

test_url = re.sub('{z}', str(zoom), test_url)
test_url = re.sub('{x}', str(tile_x), test_url)
test_url = re.sub('{y}', str(tile_y), test_url)
print(test_url)

-14.812410006298874 -168.38463752739523 8
8 138
http://127.0.0.1:8000/tiles/WebMercatorQuad/8/8/138@1x?url=https%3A%2F%2Fearth-search.aws.element84.com%2Fv0%2Fcollections%2Fsentinel-s2-l2a-cogs%2Fitems%2FS2A_2LQJ_20230227_0_L2A&assets=B03&assets=B11&algorithm=detectFlood&rescale=-1%2C1&colormap_name=viridis


In [27]:
# Note: Tile Server rescales uint16 data auto to max/min and shows black/white tiles 
bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=r["minzoom"] + 1
)

aod_layer = TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Swisstopo"
)
aod_layer.add_to(m)
m

In [ ]:
#titiler_endpoint = "https://titiler.xyz"
titiler_endpoint = "http://127.0.0.1:8000"
#'http://127.0.0.1:8000/search/?country=afghanistan'
#url = "https://data.geo.admin.ch/ch.swisstopo.swissalti3d/swissalti3d_2019_2573-1085/swissalti3d_2019_2573-1085_0.5_2056_5728.tif"
url = "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2A_30TVT_20221112_0_L2A"
r = httpx.get(
    #f"{titiler_endpoint}/cog/tilejson.json",
    f"{titiler_endpoint}/tilejson.json",
    params = {
        #"country": "afghanistan",
        "url": url,
        "assets": ["green", "swir16"],
        #"assets": ["green", "blue"],
        'minzoom': 8,
        'maxzoom': 16,
        "algorithm": "MNDWI",
        #"buffer": 3,
    }
).json()
r
test_url = r['tiles'][0]
deg_y, deg_x, zoom = r['center']
print(test_url)
print(deg_x, deg_y, zoom)

In [ ]:
#titiler_endpoint = "https://titiler.xyz"
#titiler_endpoint = "http://127.0.0.1:8000"
titiler_endpoint = "http://127.0.0.1:8000/search/?country=afghanistan"
#'http://127.0.0.1:8000/search/?country=afghanistan'
#url = "https://data.geo.admin.ch/ch.swisstopo.swissalti3d/swissalti3d_2019_2573-1085/swissalti3d_2019_2573-1085_0.5_2056_5728.tif"
#url = "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2A_30TVT_20221112_0_L2A"
r = httpx.get(
    #f"{titiler_endpoint}/cog/tilejson.json",
    #f"{titiler_endpoint}/tilejson.json",
    titiler_endpoint,
    follow_redirects=True
    #params = {
    #    "country": "afghanistan",
        #"url": url,
        #"assets": ["green", "swir16"],
        #'minzoom': 8,
        #'maxzoom': 16,
        #"algorithm": "MNDWI",
        #"buffer": 3,
    #}
).json()
#print(dir(r))
print(r)
#print(r)
#test_url = r['tiles'][0]
#deg_y, deg_x, zoom = r['center']
#print(test_url)
#print(deg_x, deg_y, zoom)


In [ ]:
titiler_endpoint = "https://titiler.xyz"
#urls = ['https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_2LMJ_20230225_0_L2A',
# 'https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_2LNJ_20230225_0_L2A']
#urls = 'https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_2LMJ_20230225_0_L2A, https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_2LNJ_20230225_0_L2A'
#urls = ['https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/2/L/MJ/2023/2/S2B_2LMJ_20230225_0_L2A/B03.tif',
# 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/2/L/NJ/2023/2/S2B_2LNJ_20230225_0_L2A/B03.tif']
#urls = 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/2/L/MJ/2023/2/S2B_2LMJ_20230225_0_L2A/B03.tif, https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/2/L/NJ/2023/2/S2B_2LNJ_20230225_0_L2A/B03.tif'
#urls = "data/B03.tif, data/B03_2.tiff"
urls = "/Users/sashkawarner/repos/flood-data-api/app/data/B03.tiff,/Users/sashkawarner/repos/flood-data-api/app/data/B03_2.tiff" #NO SPACE
#titiler_endpoint = "http://127.0.0.1:8000"
r = httpx.get(
    f"{titiler_endpoint}/mosaic/tilejson.json",
    params={
        # For this demo we are use the same mosaic but stored on the web
        #"url": "./data/test_mosaic.json",
        "url": urls,
        "minzoom": 8,
        "maxzoom": 16,
        #"assets": ["B03"],
        #"bounds": [-171.26452474589144, -15.46498932556128, -170.9090253707599,-14.472279246488112]
        "bounds": [117, 121, 117, 121]
    }
).json()
print(r)

In [ ]:
from pygeotile.point import Point
import numpy as np
import math
import re
bounds = [-171.26452474589144, -15.46498932556128, -170.9090253707599,-14.472279246488112]

test_url = r['tiles'][0]
deg_y, deg_x, zoom = r['center']
print(test_url)
print(deg_x, deg_y, zoom)

def deg2num(lat_deg, lon_deg, zoom):
  lat_rad = math.radians(lat_deg)
  n = 2.0 ** zoom
  xtile = int((lon_deg + 180.0) / 360.0 * n)
  ytile = int((1.0 - math.asinh(math.tan(lat_rad)) / math.pi) / 2.0 * n)
  return (xtile, ytile)

tile_x, tile_y = deg2num(lat_deg=deg_x, lon_deg=deg_y, zoom=zoom)
print(tile_x, tile_y)
b_x, b_y = deg2num(bounds[0], bounds[1], 8)
b_x2, b_y2 = deg2num(bounds[2], bounds[3], 8)

print(b_x, b_y, b_x2, b_y2)

In [ ]:
test_url = re.sub('{z}', str(zoom), test_url)
test_url = re.sub('{x}', str(tile_x), test_url)
test_url = re.sub('{y}', str(tile_y), test_url)
print(test_url)